Get all Wikidata objects and checks if they are connected to an OSM object using an [API](https://osm.wikidata.link/tagged)

* this [Notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/OSM_Wikidata.ipynb)

* API [Wikidata to OpenStreetMap](https://osm.wikidata.link/tagged)
  * eg. [https://osm.wikidata.link/tagged/api/item/Q106708773](https://osm.wikidata.link/tagged/api/item/Q106708773)

* Another tool [osm.wikidata.link](https://osm.wikidata.link/search)
  
Status:  



| Date | Outdoor Gym | no WD - OSM | 
| ------------- |:-------------:| -----:|


TODO: 
* 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-08-01 11:25:49.638577


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/3Uni
queryLighthouse = """SELECT DISTINCT (REPLACE(STR(?site), ".*Q", "Q") AS ?qid) ?site ?siteLabel ?coordinates ?svWikipedia  WHERE {
  #?site wdt:P17 wd:Q34.
  ?site wdt:P31/wdt:P279* wd:Q41176.
  ?site wdt:P625 ?coordinates.
   minus {
    { ?site wdt:P10689 ?OSMid. }
    UNION
    { ?site wdt:P402 ?OSMrelid. }
    UNION 
    { ?site wdt:P11693 ?OSMnode. }
  }
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }  {
#    ?svWikipedia schema:about ?site;
#      schema:inLanguage "sv";
#      schema:isPartOf <https://sv.wikipedia.org/>.}
}"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDo = get_sparql_dataframe(endpoint_url, queryLighthouse )
WDo["Source"] = "WD"     
WDo.shape

(11655, 6)

In [4]:
WDo.head()

,qid,site,siteLabel,coordinates,svWikipedia,Source
0,Q307336,http://www.wikidata.org/entity/Q307336,None,Point(17.218267 59.2592),None,WD
1,Q388667,http://www.wikidata.org/entity/Q388667,None,Point(11.989166666 57.695555555),None,WD
2,Q478165,http://www.wikidata.org/entity/Q478165,None,Point(13.19833333 55.71166667),None,WD
3,Q995525,http://www.wikidata.org/entity/Q995525,None,Point(15.11758 56.22542),None,WD
4,Q206435,http://www.wikidata.org/entity/Q206435,None,Point(12.976388888 55.613333333),None,WD


In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWDo = []
for WD, row in tqdm(WDo.iterrows(), total=WDo.shape[0]):
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qid"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
    #print (r.status)
    try:
        #print(data)
        osmid = data["osm"][0]["id"]            
        osmtype = data["osm"][0]["type"]              
    except:
        #print ("error")
        #print(data)
        osmid =""
        osmtype =""
    new_item['osmid'] = osmid  
    new_item['type'] = osmtype 
    
    listWDo.append(new_item)
print (len(listWDo))

100%|██████████| 11655/11655 [30:16<00:00,  6.42it/s] 

11655


In [6]:
OSMtot = pd.DataFrame(listWDo,
                  columns=['wikidata','type','osmid'])
OSMtot.shape


(11655, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,type,osmid
0,Q307336,,
1,Q388667,way,28219722
2,Q478165,,
3,Q995525,,
4,Q206435,way,30926877
5,Q581004,node,312709005
6,Q929199,way,304835599
7,Q289298,,
8,Q781879,,
9,Q499733,node,722059523


In [8]:
#OSMempty = OSMtot.osmid.notnull()
OSMtot[(OSMtot['osmid']=="")].shape

(10168, 3)

In [9]:
OSMEmpty =OSMtot[(OSMtot['osmid']=="")]

In [10]:
OSMEmpty.shape

(10168, 3)

In [11]:
OSMEmpty.to_csv("WD - OSM byggnad missingSwe.csv")

OSMEmpty.head()

,wikidata,type,osmid
0,Q307336,,
2,Q478165,,
3,Q995525,,
7,Q289298,,
8,Q781879,,


In [12]:
OSMConnected=OSMtot[(OSMtot['osmid']!="")]
OSMConnected.to_csv("WD - OSM_byggnadSwe.csv")
OSMConnected.head()

,wikidata,type,osmid
1,Q388667,way,28219722
4,Q206435,way,30926877
5,Q581004,node,312709005
6,Q929199,way,304835599
9,Q499733,node,722059523


In [13]:
print("*", start_time.strftime("%Y%m%d"),"WD objects", WDo.shape[0], "ej OSM kopplade",OSMEmpty.shape[0]) 


* 20230801 WD objects 11655 ej OSM kopplade 10168


Generate Markdown table eg.
| 20210526     | 2802 | 2050 |1147 | 254 | 213| 84|


In [14]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      WDo.shape[0],"|",OSMEmpty.wikidata.nunique(),"|")


| 20230801 | 11655 | 10133 |


In [15]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-08-01 11:57:04.075199
Time elapsed (hh:mm:ss.ms) 0:31:14.437254
